In [5]:
import pandas as pd
import numpy as np
import pytz, sys,os
import plotly.express as px

sys.path.append(os.getcwd())

from datetime import datetime as dt
from datetime import timezone
from dateutil import tz
from src.config import SUBSCRIPTION_KEY, CUSTOMER_ID
from PF_API_Shared import get_all_plants, get_plant_devices, Dispatch,TimeInterval
from API_Intergration import plant_tz


In [2]:

plant = 'Palmer'
start = dt(2022,9,1)
end = dt(2022,9,30)


In [3]:
dev_tags = [("Combiner", ["DC_CURRENT"]),
            ("Inverter", ["DC_CURRENT"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),
                ("Meter",["AC_POWER"])]
  
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)



In [4]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

In [6]:
ident = [id for site, type, id, name in data.columns]
data.columns = ident


In [7]:
data = data.set_index([data.index])
data.index = pd.to_datetime(data.index)
data = data[~data.index.duplicated(keep='first')]
_tz = plant_tz(plant)
data.index = data.index.tz_convert(_tz)


In [8]:
"""For Filtered data, the data is filtered by the following parameters
        a. POA > 100
        b. (Meter Power > 1.5 MW) & (Meter Power < 25 MW)
        c. Hour > 12 (For shading purposes)
        d. All CBX's < 0 = 0
"""

data.loc[:, "Meter_Power"] = data.filter(regex="MTR").median(axis=1)
data.loc[:, "POA"] = data.filter(regex="MET").median(axis=1)

curtailment_limit = 499999

new_df = data.between_time("7:00", "17:00")
new_df = data[(data.loc[:,"POA"] > 100)&(data.loc[:,"Meter_Power"]<curtailment_limit*0.99)]
new_df[new_df<=0.00] = 0
day_df = new_df.resample("1D").sum()

day_df


C:\Users\dpinales\AppData\Local\Temp\ipykernel_96756\4223233658.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_df<=0.00] = 0
C:\Users\dpinales\AppData\Local\Temp\ipykernel_96756\4223233658.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_df<=0.00] = 0


,2096.05.011.BLK01.PAD01.INV01.CMB01,2096.05.011.BLK01.PAD01.INV01.CMB02,2096.05.011.BLK01.PAD01.INV01.CMB03,2096.05.011.BLK01.PAD01.INV01.CMB04,2096.05.011.BLK01.PAD01.INV01.CMB05,2096.05.011.BLK01.PAD01.INV01.CMB06,2096.05.011.BLK01.PAD01.INV01.CMB07,2096.05.011.BLK01.PAD01.INV01.CMB08,2096.05.011.BLK01.PAD01.INV01.CMB09,2096.05.011.BLK01.PAD01.INV01.CMB10,...,2096.05.011.BLK01.PAD02.INV01,2096.05.011.MET09.PYR01,2096.05.011.MET10.PYR01,2096.05.011.MET11.PYR01,2096.05.011.MET13.PYR01,2096.05.011.MET17.PYR01,2096.05.011.METREF01.PYR01,2096.05.011.SWG01.MTR01,Meter_Power,POA
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-01 00:00:00-04:00,21106.4,18442.2,19935.6,19738.0,16675.0,21319.6,18192.0,19091.8,16391.2,14872.0,...,311068.440169,58252.893333,0.0,0.0,113713.590000,102932.920450,115255.100268,536163.633139,536163.633139,112941.929447
2022-09-02 00:00:00-04:00,18962.2,16631.4,17847.8,17928.2,14977.6,18982.8,16642.0,17385.8,14748.0,13473.8,...,278700.020090,63732.926667,0.0,0.0,101848.570000,91784.429695,103110.669826,488075.831757,488075.831757,100297.682476
2022-09-03 00:00:00-04:00,12870.4,11758.0,11882.0,12059.4,10559.4,13015.4,11082.2,12125.8,11455.2,10380.8,...,190843.379388,50404.443333,0.0,0.0,72494.016667,65517.732637,66667.849340,350283.038519,350283.038519,67910.847302
2022-09-04 00:00:00-04:00,12783.2,10989.6,11716.6,12058.4,10134.8,12656.2,11251.6,11871.0,9792.8,9145.6,...,184716.639807,32194.910000,0.0,0.0,66722.926667,60610.767741,69059.888936,332001.398132,332001.398132,65227.005389
2022-09-05 00:00:00-04:00,8096.2,7391.2,7403.6,7688.8,6674.6,8132.6,7153.2,7757.6,7012.4,6484.4,...,120788.459961,24101.966667,0.0,0.0,42686.036667,39372.519287,43088.611711,227678.340143,227678.340143,41244.924717
2022-09-06 00:00:00-04:00,13303.4,12025.2,12357.4,12692.6,11019.6,13520.6,11694.0,12762.6,11760.0,10618.2,...,197370.380231,49450.783333,0.0,0.0,73637.580000,66564.465983,73925.672131,359971.756104,359971.756104,70745.885666
2022-09-07 00:00:00-04:00,9866.2,9004.4,9053.8,9577.4,8418.6,10072.8,8903.4,9876.8,8359.4,8278.8,...,146858.240216,37896.480000,0.0,0.0,53463.606667,48030.954399,50128.837050,274475.954108,274475.954108,50235.003860
2022-09-08 00:00:00-04:00,5754.4,5408.8,5275.0,5247.6,4806.8,5858.2,4831.8,5530.4,4730.4,4670.0,...,84851.600143,16732.553333,0.0,0.0,30809.890000,28842.303891,30641.219972,163549.316049,163549.316049,29702.349165
2022-09-09 00:00:00-04:00,18597.6,16397.0,17324.4,17362.4,14878.0,18639.8,15950.4,15908.8,12895.8,12899.8,...,265234.179907,52290.856667,0.0,0.0,97376.743333,88320.028693,100234.070544,474882.369156,474882.369156,94362.945560


In [11]:
path = "Project_Metadata_tables.xlsx"

dc_meta = pd.read_excel(path, sheet_name="Combiner_table").filter(items= ['plant_name', 'string_count','DC Rating Power kW', 'Module IMP'], axis=1)


In [20]:
day_df = day_df.filter(regex="CMB")

dc_issues = dc_meta[dc_meta["plant_name"]==plant]  
dc_rating = [i for i in dc_issues['DC Rating Power kW'].values]

relative_dc = []

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)

string_count = [s for s in dc_issues['string_count']]

cmb_relative = day_df.div(relative_dc)

cmb_relative.loc[:,'norm'] = cmb_relative.apply(pd.Series.nlargest, axis = 1, n = 5).median(axis = 1)
t = cmb_relative.div(cmb_relative['norm'], axis = 0).clip(lower = 0, upper = 1)
t = t.iloc[:,:-1]

down_strings = (string_count*(1-t)).round()

print(down_strings.index.values)
# down_strings.to_csv('latitude_DC.csv')
down_strings

['2022-09-01T04:00:00.000000000' '2022-09-02T04:00:00.000000000'
 '2022-09-03T04:00:00.000000000' '2022-09-04T04:00:00.000000000'
 '2022-09-05T04:00:00.000000000' '2022-09-06T04:00:00.000000000'
 '2022-09-07T04:00:00.000000000' '2022-09-08T04:00:00.000000000'
 '2022-09-09T04:00:00.000000000' '2022-09-10T04:00:00.000000000'
 '2022-09-11T04:00:00.000000000' '2022-09-12T04:00:00.000000000'
 '2022-09-13T04:00:00.000000000' '2022-09-14T04:00:00.000000000'
 '2022-09-15T04:00:00.000000000' '2022-09-16T04:00:00.000000000'
 '2022-09-17T04:00:00.000000000' '2022-09-18T04:00:00.000000000'
 '2022-09-19T04:00:00.000000000' '2022-09-20T04:00:00.000000000'
 '2022-09-21T04:00:00.000000000' '2022-09-22T04:00:00.000000000'
 '2022-09-23T04:00:00.000000000' '2022-09-24T04:00:00.000000000'
 '2022-09-25T04:00:00.000000000' '2022-09-26T04:00:00.000000000'
 '2022-09-27T04:00:00.000000000' '2022-09-28T04:00:00.000000000'
 '2022-09-29T04:00:00.000000000']


,2096.05.011.BLK01.PAD01.INV01.CMB01,2096.05.011.BLK01.PAD01.INV01.CMB02,2096.05.011.BLK01.PAD01.INV01.CMB03,2096.05.011.BLK01.PAD01.INV01.CMB04,2096.05.011.BLK01.PAD01.INV01.CMB05,2096.05.011.BLK01.PAD01.INV01.CMB06,2096.05.011.BLK01.PAD01.INV01.CMB07,2096.05.011.BLK01.PAD01.INV01.CMB08,2096.05.011.BLK01.PAD01.INV01.CMB09,2096.05.011.BLK01.PAD01.INV01.CMB10,...,2096.05.011.BLK01.PAD02.INV01.CMB07,2096.05.011.BLK01.PAD02.INV01.CMB08,2096.05.011.BLK01.PAD02.INV01.CMB09,2096.05.011.BLK01.PAD02.INV01.CMB10,2096.05.011.BLK01.PAD02.INV01.CMB11,2096.05.011.BLK01.PAD02.INV01.CMB12,2096.05.011.BLK01.PAD02.INV01.CMB13,2096.05.011.BLK01.PAD02.INV01.CMB14,2096.05.011.BLK01.PAD02.INV01.CMB15,2096.05.011.BLK01.PAD02.INV01.CMB16
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-01 00:00:00-04:00,2.0,6.0,1.0,2.0,5.0,0.0,4.0,3.0,6.0,7.0,...,2.0,0.0,4.0,2.0,2.0,2.0,3.0,1.0,0.0,12.0
2022-09-02 00:00:00-04:00,2.0,6.0,0.0,1.0,5.0,0.0,3.0,2.0,5.0,7.0,...,2.0,0.0,4.0,2.0,2.0,2.0,4.0,1.0,0.0,11.0
2022-09-03 00:00:00-04:00,2.0,6.0,1.0,2.0,5.0,0.0,4.0,2.0,3.0,5.0,...,2.0,0.0,4.0,2.0,1.0,2.0,4.0,1.0,0.0,9.0
2022-09-04 00:00:00-04:00,2.0,6.0,1.0,1.0,5.0,0.0,3.0,1.0,5.0,7.0,...,3.0,0.0,4.0,1.0,1.0,2.0,4.0,2.0,0.0,10.0
2022-09-05 00:00:00-04:00,2.0,5.0,1.0,1.0,4.0,0.0,3.0,1.0,3.0,5.0,...,2.0,0.0,4.0,0.0,1.0,2.0,4.0,1.0,0.0,8.0
2022-09-06 00:00:00-04:00,2.0,6.0,1.0,1.0,4.0,0.0,3.0,1.0,3.0,5.0,...,2.0,0.0,3.0,0.0,1.0,2.0,4.0,1.0,0.0,9.0
2022-09-07 00:00:00-04:00,2.0,6.0,1.0,1.0,4.0,0.0,3.0,0.0,4.0,4.0,...,2.0,0.0,4.0,1.0,1.0,2.0,4.0,2.0,0.0,8.0
2022-09-08 00:00:00-04:00,4.0,6.0,3.0,4.0,5.0,1.0,5.0,3.0,6.0,6.0,...,3.0,0.0,5.0,0.0,2.0,4.0,5.0,3.0,0.0,10.0
2022-09-09 00:00:00-04:00,2.0,6.0,1.0,2.0,5.0,0.0,3.0,4.0,7.0,7.0,...,2.0,0.0,4.0,2.0,2.0,3.0,4.0,2.0,0.0,13.0


In [18]:



fig = px.imshow(down_strings, labels=dict(x="Downstrings", y="Dates"), x=down_strings.columns, y=down_strings.index.values)
fig.show()

In [ ]:
e_downstrings = sum(int(down_strings[:]))
total_string = sum(int(string_count))
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

In [ ]:
sum_of_cbx = cbx_data["Sum_of_CBX"]
sum_of_inv = inv_df["Sum_of_inv"]

def cmb_summation(a,b):
    total = (a/b)
    return total


In [ ]:
# cbx_data.to_csv(f"{plant} - August_Daily_Derates.csv")
cbx_data.to_csv(f"{plant} - API Data.csv")